In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [23]:
import random
import string
import nltk
import spacy
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sqlalchemy import create_engine

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import gutenberg

from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
from chatterbot.conversation import Statement

import warnings

warnings.filterwarnings(action="ignore")

<IPython.core.display.Javascript object>

In [3]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "cornell_movie_dialogs"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)

dialogs_df = pd.read_sql_query("select * from dialogs", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


dialogs_df.head(10)

,index,dialogs
0,0,Can we make this quick? Roxanne Korrine and A...
1,1,"Well, I thought we'd start with pronunciation,..."
2,2,Not the hacking and gagging and spitting part....
3,3,Okay... then how 'bout we try out some French ...
4,4,You're asking me out. That's so cute. What's ...
5,5,Forget it.
6,6,"No, no, it's my fault -- we didn't have a prop..."
7,7,Cameron.
8,8,"The thing is, Cameron -- I'm at the mercy of a..."
9,9,Seems like she could get a date easy enough...


<IPython.core.display.Javascript object>

In [16]:
diag_samp = dialogs_df.sample(frac=0.25)

<IPython.core.display.Javascript object>

In [17]:
nlp = spacy.load("en", disable=["parser", "ner"])
nlp.add_pipe(nlp.create_pipe("sentencizer"))
nlp.max_length = 20000000
dialog = " ".join(diag_samp.dialogs)
dialog_doc = nlp(dialog)

<IPython.core.display.Javascript object>

In [18]:
dialog_sents = [sent.text for sent in dialog_doc.sents if len(sent.text) > 1]
dialog_sents

["Since he and his wife have been in custody, they've killed--- I can't help it if I'm stupid.",
 'Why?',
 "Because that's the kind of man Bill Parrish is.",
 "I think you're a lying, back-stabbing psycho bitch, and one day you'll kill me.",
 "But it's nice to know you cared... Gee, thanks!",
 'What?',
 'Well, I understand the problem about the supermarkets, but I was talking to this girl last night, and she made the point that a likeable personality is a valuable business asset.',
 'Bad.',
 'Sent to the City Hospital.',
 'They ruptured his hernia again.',
 'Second verse!',
 'What was he doing?',
 'I gonna meet that family of yours?',
 'I call him the boy with the ice cream face!',
 'Son of a bitch!',
 'Her name is Alice.',
 'You know I love that girl to death.',
 "If I ever found out somebody was makin' out with her, I'd take this...  ...and shove it so far up his ass it would come out his mouth... Then you know what?...",
 "Um...who's that?",
 'They decided to no-file.',
 "Tell me wh

<IPython.core.display.Javascript object>

In [31]:
# Create a chatbot
chatbot = ChatBot(dialog)
# This is to remove the accumulated knowledge base
chatbot.storage.drop()

# Create a new trainer for the chatbot
trainer = ListTrainer(chatbot)

# Train the chatbot based on dialog
trainer.train(dialog_sents)

List Trainer: [####################] 100%


<IPython.core.display.Javascript object>

In [32]:
GREETING_INPUTS = ["hello", "hi", "greetings", "what's up", "hey"]
GREETING_RESPONSES = ["hello", "hi", "hey", "hi there"]


def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

<IPython.core.display.Javascript object>

In [33]:
print(
    "ChatterBot: I will try to respond to you reasonably. If you want to exit, type bye."
)

# Below is the chatting
while True:

    user_input = input("User: ")
    user_input = user_input.lower()

    if user_input != "bye":
        if user_input == "thanks" or user_input == "thank you":
            break
            print("ChatterBot: You're welcome.")
        else:
            if greeting(user_input) != None:
                print("ChatterBot: " + greeting(user_input))
            else:
                print("ChatterBot: ", end="")
                print(chatbot.get_response(user_input).text)
    else:
        print("ChatterBot: Bye! It was a great chat.")
        break

ChatterBot: I will try to respond to you reasonably. If you want to exit, type bye.
User: hey
ChatterBot: hi there
User: what are you doing
ChatterBot: I gonna meet that family of yours?
User: maybe. I feel like it's abit soon for that htough
ChatterBot: And somehow I knew something horrible was going to happen, even before it did.
User: well what do you expect. we've been dating for a monthj
ChatterBot: So don't embarrass me.
User: how am I embarassing you
ChatterBot: Oh, we'll have to do better than that.
User: let's change the subject
ChatterBot: You ever been with a woman?
User: yes
ChatterBot: I used to hate it.
User: how do you feel about propane?
ChatterBot: Monsieur Rick!
User: I believe his name is hank but ok
ChatterBot: I know it all seems crazy, and maybe it did start out, you know, for the... for the sex and the danger.
User: you are quite combative
ChatterBot: I gonna meet that family of yours?
User: lol nah
ChatterBot: Then why not just fuckin' say better than expected?


<IPython.core.display.Javascript object>

This chatbot is actually pretty entertaining. Would be interesting to train it on the whole set but I simply don't have the patience for that.